In [1]:
from keras.preprocessing import sequence
import pandas as pd
import numpy as np

/home/huypham/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('train_encode.csv')

In [3]:
df.head()

,descVec,wordIdVec
0,"[2427, 3953, 4687, 2416, 2337, 772, 40, 4878, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[2427, 3209, 3954, 4687, 0, 3730, 4945, 40, 12...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[2427, 3209, 3954, 4687, 2195, 2433, 3324, 259...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[2427, 5288, 4809, 4093, 5335, 3253, 872, 3324...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[2799, 2031, 380, 3412, 772, 183, 40, 2061, 24...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
max([len(i) for i in df.descVec])

270

In [5]:
maxlen = 500
max_features = 6000

In [59]:
input_train = [list(map(int, i.strip('[').strip(']').split(','))) for i in df.descVec]
y_train = [list(map(int, i[1:-1].replace(', ', ' ').split(' '))) for i in df.wordIdVec]
input_train_pad = sequence.pad_sequences(input_train, maxlen=maxlen, padding='post')
y_train_pad = sequence.pad_sequences(y_train, maxlen=maxlen, padding='post')

In [161]:
testdf = pd.read_csv('test_encode.csv')

input_test = [list(map(int, i.strip('[').strip(']').split(','))) for i in testdf.descVec]
y_test = [list(map(float, i[1:-1].replace(', ', ' ').split(' '))) for i in testdf.wordIdVec]
input_test_pad = sequence.pad_sequences(input_test, maxlen=maxlen, padding='post')
y_test_pad = sequence.pad_sequences(y_test, maxlen=maxlen, padding='post')

In [60]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

In [142]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(500, activation='sigmoid'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics =['acc'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_8 (Dense)              (None, 500)               16500     
Total params: 344,820
Trainable params: 344,820
Non-trainable params: 0
_________________________________________________________________


In [143]:
res = model.fit(input_train_pad, y_train_pad, epochs = 10, batch_size=128, validation_split=0.2)

Train on 7719 samples, validate on 1930 samples
Epoch 1/10
7719/7719 [==============================] - 26s 3ms/step - loss: 5.4905 - acc: 0.0961 - val_loss: 4.1550 - val_acc: 0.1047
Epoch 2/10
7719/7719 [==============================] - 25s 3ms/step - loss: 3.3967 - acc: 0.0928 - val_loss: 2.9136 - val_acc: 0.1047
Epoch 3/10
7719/7719 [==============================] - 27s 3ms/step - loss: 2.8114 - acc: 0.1095 - val_loss: 2.6912 - val_acc: 0.3187
Epoch 4/10
7719/7719 [==============================] - 26s 3ms/step - loss: 2.6717 - acc: 0.3342 - val_loss: 2.6044 - val_acc: 0.3187
Epoch 5/10
7719/7719 [==============================] - 31s 4ms/step - loss: 2.6156 - acc: 0.3342 - val_loss: 2.5606 - val_acc: 0.3187
Epoch 6/10
7719/7719 [==============================] - 31s 4ms/step - loss: 2.5718 - acc: 0.3342 - val_loss: 2.5275 - val_acc: 0.3187
Epoch 7/10
7719/7719 [==============================] - 30s 4ms/step - loss: 2.5477 - acc: 0.3342 - val_loss: 2.5097 - val_acc: 0.3187
Epoch 8

In [162]:
input_test_pad.shape

(2413, 500)

In [163]:
y_test_pred = model.predict(input_test_pad)

In [165]:
y_test_pred.shape

(2413, 500)

In [167]:
y_test_pad.shape

(2413, 500)

In [177]:
y_test_pred_nopad = [y_test_pred[:len(i)] for i in input_test]

In [147]:
#list(y_test_pred_nopad[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [178]:
#from sklearn.metrics import accuracy_score
acc=0
for i, j in zip(list(y_test), list(y_test_pred_nopad)):
    if list(i)==list(j):
        acc += 1
        
print(acc/len(y_test_pad))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
su = 0
for i in y_test_pad:
    if sum(i) != 0:
        su += 1
print(su)

In [ ]:
# 0.14022178464089544

In [180]:
y_test_pred_nopad.shape

AttributeError: 'list' object has no attribute 'shape'

In [173]:
[0,0] == [0,0]

True

# Y TEST pred

In [ ]:
import pandas as pd
import csv
from utils import *

vocab = pd.read_csv('vocab.csv', na_filter = False)
vocab = list(vocab.vocab)

df = pd.read_csv('test.csv', index_col=0)
df.desc = [textPrep(i) for i in df.desc]
df.wordId = [textPrep(i) for i in df.wordId]

wordIdVec_rev = [getWordIdenFromOutputVector(originSt=list(df.desc)[i], outVec=y_test_pad[i]) for i in range(len(wordIdVec))]